In [143]:
import json
from web3 import Web3
import os
import platform
from getpass import getpass
import random

In [144]:
operating_system = platform.system()
d = '/'
if operating_system == 'Windows':
    d = '\\'

ganache_url = "HTTP://127.0.0.1:7545"
web3 = Web3(Web3.HTTPProvider(ganache_url))

# This is our main account that controls everything
web3.eth.defaultAccount = web3.eth.accounts[0]

In [145]:
web3.eth.accounts

['0xb6ef75b96D856a6cE0C796be4a7ABfB81E5D600A',
 '0xa2D0B6dbCd04a789da44CCECA9d66B6dfc93A124',
 '0xEE4F5078d802207d68eE9668AE6b82FCFCF7e290',
 '0x9cbFF84198C1c3ed898C52Fbc0076Bf008476C4B',
 '0x18a31E094820e4C9C79a073A0927FA72635b18C7',
 '0xDED00A7f19D445B2c6D3ddFffbf78BB490771937',
 '0x37594b8C8aC1D94457C86e30dE793099CE659817',
 '0x6a1ab8E6bb7e9CC3463997FC7F38e3B3855282b5',
 '0xAfd3bB14ff7735B0D9792e5bcc9BB5e758a32ccB',
 '0x8b23fa43919cB11b9CDa04FA45B92aaA0A0Adcd8']

In [146]:
def getContract(filename):
    with open('..'+ d +'build'+ d +'contracts'+ d +filename) as contractFile:
        contractJson=json.load(contractFile)
        contractABI =contractJson['abi']
        contractAdd=web3.toChecksumAddress(contractJson['networks']['5777']['address']) 
        return contractABI, contractAdd



# Get the organization contract address
organizationContractABI, organizationContractAdd = getContract('Organization.json')	
organizationContract = web3.eth.contract(address=organizationContractAdd, abi=organizationContractABI)

#get the user contract address
userContractABI,userContractAdd = getContract('User.json')
userContract = web3.eth.contract(address=userContractAdd, abi=userContractABI)

# Get the accounts contract address
accountsContractABI,accountsContractAdd = getContract('Accounts.json')
accountsConract = web3.eth.contract(address=accountsContractAdd, abi=accountsContractABI)

loansContractABI, loansContractAddress= getContract('Loans.json')


In [147]:
# This should be changed to be a more robus way. We can give the loansContractAddress using the constructor
# but this has problems when doing this in the 2_deploy_contracts.js
userContract.functions.setLoansContractAddress(loansContractAddress).transact()
organizationContract.functions.setLoansContractAddress(loansContractAddress).transact()

HexBytes('0x1e6392a418c62c5292bcc0f5d7b5262a862e5d7040cda51dac5993b9fc83bd07')

HexBytes('0xedb695de5c7ff7c42f04259123c2508e0347e5627e9a52be7b7f5c0218ad13de')

In [148]:
accountsConract.functions.add(web3.eth.accounts[1], False).transact()
accountsConract.functions.add(web3.eth.accounts[2], False).transact()
accountsConract.functions.add(web3.eth.accounts[5], True).transact()
accountsConract.functions.add(web3.eth.accounts[6], True).transact()

HexBytes('0x60457ad53fa0c7f3218920fd1e3997adbeacf21fa61d54389c85cdadc66831ad')

HexBytes('0x73edf025c4bc5181ecfcafd0bf295580deaa1f4f2be130904295e2f891d1aeb2')

HexBytes('0xe72958c362c98c0a3456307d8d6a3243c0512d47a96f6f544c7f94dac5602989')

HexBytes('0x09f88aa767c9118898a5a81134d4c4fe07bbc26da0d0e785e873a8276912f159')

In [149]:
def deleteUser():
    accountsConract.functions.deleteAccount(web3.eth.accounts[1]).transact()

def createLoan(_address,_loaner,_amount,_loansContractAddress):
    check=organizationContract.functions.createLoan(_address,_loaner,_amount).transact()
    return check

In [150]:
deleteUser()

In [151]:
def getPendingLoans(_userContract, _loanieAddress):
    transaction = _userContract.functions.getPendingLoans(
    ).buildTransaction({
    'gas': 70000,
    'gasPrice': web3.toWei('1', 'gwei'),
    'from': _loanieAddress,
    'nonce': web3.eth.getTransactionCount(_loanieAddress)
    }) 
    private_key = getpass('Enter your key: ')
    signed_txn = web3.eth.account.signTransaction(transaction, private_key=private_key)
    transaction_hash = web3.eth.sendRawTransaction(web3.toHex(signed_txn.rawTransaction))
    receipt = web3.eth.getTransactionReceipt(transaction_hash)
    rich_logs = _userContract.events.getAmounts().processReceipt(receipt)
    event_values = rich_logs[0]['args'] # Dictionary
    return event_values


In [165]:
# Suppose bank wants to create a loan
loanieAddress = input("Enter the addrses: ")

### Check to see whether it is a user or organization address
index = accountsConract.functions.getIndex(loanieAddress).call()

Enter the addrses: 0x37594b8C8aC1D94457C86e30dE793099CE659817


In [161]:
index

1

In [162]:
# Get pending loans
pendingLoans = [] # 'amount loanerAddress id'
if index != -1:
    loanieType = accountsConract.functions.getType(index).call()
    if not loanieType:
        #res = organizationContract.functions.createLoan(loanieAddress).call()
        values = getPendingLoans(userContract, loanieAddress)
        for i in range(len(values['_amounts'])):
            string = ''
            for key in values:
                string += str(values[key][i]) + ' '
            print(string)
            pendingLoans.append(string)
            
else:
    print("This account is not registered in our system.")

Enter your key: ········
1603 0x37594b8C8aC1D94457C86e30dE793099CE659817 1588773649 
53772 0xDED00A7f19D445B2c6D3ddFffbf78BB490771937 1588773689 
99000 0xDED00A7f19D445B2c6D3ddFffbf78BB490771937 1588773697 


In [164]:
# Confirm or reject loans
private_key = getpass('Enter your key: ')
for pendingLoan in pendingLoans:
    print('for address: ' + pendingLoan.split(' ')[1])
    loanId = pendingLoan.split(' ')[2]
    choice = input('c/r?')
    if choice == 'c':
        transaction = userContract.functions.validateLoan(True,
        int(loanId)).buildTransaction({
        'gas': 70000,
        'gasPrice': web3.toWei('1', 'gwei'),
        'from': loanieAddress,
        'nonce': web3.eth.getTransactionCount(loanieAddress)
        }) 
        signed_txn = web3.eth.account.signTransaction(transaction, private_key=private_key)
        transaction_hash = web3.eth.sendRawTransaction(web3.toHex(signed_txn.rawTransaction))
    elif choice == 'r':
        transaction = userContract.functions.validateLoan(False,
        int(loanId)).buildTransaction({
        'gas': 70000,
        'gasPrice': web3.toWei('1', 'gwei'),
        'from': loanieAddress,
        'nonce': web3.eth.getTransactionCount(loanieAddress)
        }) 
        signed_txn = web3.eth.account.signTransaction(transaction, private_key=private_key)
        transaction_hash = web3.eth.sendRawTransaction(web3.toHex(signed_txn.rawTransaction))

Enter your key: ········
for address: 0x37594b8C8aC1D94457C86e30dE793099CE659817
c/r?r
for address: 0xDED00A7f19D445B2c6D3ddFffbf78BB490771937
c/r?r
for address: 0xDED00A7f19D445B2c6D3ddFffbf78BB490771937
c/r?r


In [159]:
# Create a new loan
if index != -1:
    loanieType = accountsConract.functions.getType(index).call()
    if  loanieType:
        if createLoan(web3.eth.accounts[2],loanieAddress,random.randint(1,100000),loansContractAddress):
            print("Loan created")
else:
    print("This account is not registered in our system.")

Loan created
